In [1]:
#Import all used systems
import os
import re
import xarray as xr
import netCDF4
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import hvplot.pandas
import hvplot.xarray
import hvplot.dask 
import dask
from scipy import stats
import glob
from datetime import datetime

In [2]:
#Establish Forloop variables
base_path = "/home/jovyan/ooi/kdata/RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample/" #change base path as needed
file_pattern = "*.nc"
start_year = 2025 #Files starting from this year
end_year = 2025 #Files ending in this year
num_files = 48 #Number of files to be expected - Run This block and see the amount of files filtered based on the year range

# Full search path

search_path = os.path.join(base_path, file_pattern)
print(f"Looking for files at: {search_path}")

#Extract start year from filename
def extract_start_year(filename):
    try:
        start_str = filename.split("_")[-1].split("-")[0] #20140929T190312
        return int(start_str[:4]) #Extract 2014 as the integer
    except Exception as e:
        print(f"Failed to parse year from {filename}: {e}")
        return None

#Filter and sort files
all_files = sorted(glob.glob(search_path))
print(f"Found {len(all_files)} files total")

filtered_files = []
for f in all_files:
    fname = os.path.basename(f)
    year = extract_start_year(os.path.basename(f))
    if year is not None and start_year <= year <= end_year:
        filtered_files.append(f)
    else:
        print (f"Skipping file: {fname} with year: {year}")
print(f"Filtered down to {len(filtered_files)} files")

#Select up to N files
selected_files = filtered_files[:num_files]
#Enumerate Files
for i, file_path in enumerate(selected_files):
    variable_name = f"file_{i+1}"
    print(f"{variable_name} -> {file_path}")

Looking for files at: /home/jovyan/ooi/kdata/RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample/*.nc
Found 127 files total
Skipping file: deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20140929T190312-20141125T235952.nc with year: 2014
Skipping file: deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20141126T000002-20150124T235952.nc with year: 2014
Skipping file: deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20150125T000002-20150325T235952.nc with year: 2015
Skipping file: deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20150326T000002-20150518T235952.nc with year: 2015
Skipping file: deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20150519T000002-20150729T235949.900029.nc with year: 2015
Skipping file: deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20150730T000001.900325-20150927T235948.859190.nc with year: 2015
Skipping file: deployment0001_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmps

# Build TMPSF DataSet

In [ ]:
TMPSF_vars = {}     # per-file datasets: {"TMPSF1": ds1, ...}
datasets = []       # keep order for concat
ds_all = None       # final concatenated dataset

for i, file_path in enumerate(selected_files, start=1):
    var_name = f"TMPSF{i}"          # unique key per file

    # Open + transform (INSIDE the loop)
    ds = xr.open_dataset(file_path)
    if 'obs' in ds.dims and 'time' not in ds.dims:
        ds = ds.swap_dims({'obs': 'time'})
    ds = ds.sortby('time')

    # Save uniquely
    TMPSF_vars[var_name] = ds
    datasets.append(ds)             # preserve order for concat

    print(f"{var_name} loaded & transformed from {file_path} "
          f"({str(ds.time.values[0])[:19]} → {str(ds.time.values[-1])[:19]}; rows={ds.sizes['time']})")

# ---- concat AFTER the loop ----
if datasets:
    ds_all = xr.concat(datasets, dim='time').sortby('time').chunk({'time': 1000})
    print(f"Concatenated {len(datasets)} files -> ds_all "
          f"{str(ds_all.time.min().values)[:19]} → {str(ds_all.time.max().values)[:19]} "
          f"(rows={int(ds_all.sizes['time'])})")
else:
    raise ValueError("No datasets to concatenate; 'selected_files' was empty.")


TMPSF1 loaded & transformed from /home/jovyan/ooi/kdata/RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample/deployment0002_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20250108T000005.010747-20250302T235952.181210.nc (2025-01-08T00:00:05 → 2025-03-02T23:59:52; rows=458273)
TMPSF2 loaded & transformed from /home/jovyan/ooi/kdata/RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample/deployment0002_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20250303T000002.182072-20250425T235959.314481.nc (2025-03-03T00:00:02 → 2025-04-25T23:59:59; rows=464106)
TMPSF3 loaded & transformed from /home/jovyan/ooi/kdata/RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample/deployment0002_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample_20250426T000009.312070-20250618T235959.656640.nc (2025-04-26T00:00:09 → 2025-06-18T23:59:59; rows=465112)
TMPSF4 loaded & transformed from /home/jovyan/ooi/kdata/RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample/deployment0002_RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sampl

In [ ]:
ds_all

In [ ]:
full_tmpsf=ds_all[['time', 'temperature01', 'temperature02', 'temperature03','temperature04','temperature05','temperature06','temperature07','temperature08','temperature09','temperature10','temperature11','temperature12','temperature13','temperature14','temperature15','temperature02','temperature16','temperature17','temperature18','temperature19','temperature20','temperature21','temperature22','temperature23','temperature24']].to_pandas()
full_tmpsf.tail(5)

In [ ]:
full_tmpsf.rename(columns={'temperature01':'t01','temperature02':'t02', 'temperature03':'t03'
                         ,'temperature04':'t04','temperature05':'t05','temperature06':'t06'
                         ,'temperature07':'t07','temperature08':'t08','temperature09':'t09'
                         ,'temperature10':'t10','temperature11':'t11','temperature12':'t12'
                         ,'temperature13':'t13','temperature14':'t14','temperature15':'t15'
                         ,'temperature16':'t16','temperature17':'t17','temperature18':'t18'
                         ,'temperature19':'t19','temperature20':'t20','temperature21':'t21'
                         ,'temperature22':'t22','temperature23':'t23','temperature24':'t24'}, inplace=True)
full_tmpsf.index=pd.to_datetime(full_tmpsf.index)
full_tmpsf = full_tmpsf.drop_duplicates(keep='first', inplace=False, ignore_index=False)
full_tmpsf['tl1'] = full_tmpsf[['t01', 't02', 't03', 't04', 't05', 't06', 't07']].mean(axis=1)
full_tmpsf['tl2'] = full_tmpsf[['t08', 't09', 't10', 't11', 't12', 't13', 't14']].mean(axis=1)
full_tmpsf['tl3'] = full_tmpsf[['t15', 't16', 't17', 't18', 't19', 't20', 't21']].mean(axis=1)
full_tmpsf['tl4'] = full_tmpsf[['t22', 't23', 't24']].mean(axis=1)
#full_tmpsf.drop(full_tmpsf.iloc[:, 0:25], inplace=True, axis=1)

full_tmpsf.head(5)

In [ ]:
df_tmpsf_ave = full_tmpsf.resample('d').mean()
df_tmpsf_ave.tail()

In [ ]:
#Write CSVFile
df_tmpsf_ave.to_csv('/home/jovyan/data/tmpsf_dataoutputs/tmpsf_dataoutputs/axial_tmpsf_20250108_20250910.csv')

In [ ]:
#Establish File to be read.
tmpsf_file = '/home/jovyan/data/tmpsf_dataoutputs/tmpsf_dataoutputs/axial_tmpsf_20250108_20250910.csv'

In [ ]:
#Establish Numerical values
df_tmpsf=pd.read_csv(tmpsf_file, dtype=object) #Reads the CSV
df_tmpsf.index=pd.to_datetime(df_tmpsf['time'].values)

#Looping through t01...t24 converting each column to a numeric
for i in range(1,25):
    col=f"t{i:02d}"
    if col in df_tmpsf.columns: #only converts if the column exists
        df_tmpsf[col]=pd.to_numeric(df_tmpsf[col], errors="coerce")
#del df_tmpsf['time']
df_tmpsf.head()

In [ ]:
#Creates rolling average for the thermistors.
for i in range(1,25):
    col = f"t{i:02d}"
    roll_col = f"{col}_rolling"
    if col in df_tmpsf.columns:
        df_tmpsf[roll_col]=df_tmpsf[col].rolling(window=10, min_periods=1).mean()
df_tmpsf[roll_col].head()

In [ ]:
#12x2 Array, Linear Plots: Thermistor Temperatures with Rolling Averages ------ Alter labels based on dataset
import matplotlib.pyplot as plt

fig, axs = plt.subplots(12, 2, figsize=(20, 38))

for i in range(1,25):
    row =(i-1) // 2 #row index (0-11)
    col = (i-1) % 2 #col index (0-1)
    ax = axs[row,col]

    t_col= f"t{i:02d}"
    roll_col = f"{t_col}_rolling"

    if t_col in df_tmpsf.columns:
        ax.plot(df_tmpsf.index, df_tmpsf[t_col], label=t_col, alpha=0.6)
    if roll_col in df_tmpsf.columns:
        ax.plot(df_tmpsf.index, df_tmpsf[roll_col], label=f"{t_col} Rolling Avg", linewidth=2)
    ax.set_title(f"{t_col} Temperature Over Time")
    ax.legend()

    if col ==0:
        ax.set_ylabel("Temperature (C)")
#Change these file names per dataset
plt.suptitle("Thermistor Temperatures with Rolling Averages: Jan 2025 - Sept 10th 2025", fontsize=20)
plt.tight_layout(rect=[0.05, 0, 1, 0.97])
plt.savefig("/home/jovyan/data/tmpsf_dataoutputs/tmpsf_plots/thermistortemps_jan_sept2025.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
#Histograms by Thermistor: weekly maximum temperature frequency
import matplotlib.pyplot as plt
import pandas as pd

# Ensure datetime index
df_tmpsf = df_tmpsf.copy()
df_tmpsf.index = pd.to_datetime(df_tmpsf.index, errors='coerce')

# Keep only numeric thermistor columns (skip any text/flags/etc)
therm_cols = [c for c in df_tmpsf.columns if pd.api.types.is_numeric_dtype(df_tmpsf[c])]
df_num = df_tmpsf[therm_cols]

# Weekly maxima
weekly_max = df_num.resample("W-MON").max()

# Plot histograms
fig, axs = plt.subplots(12, 2, figsize=(20, 38))

for i in range(1, 25):
    row = (i-1) // 2
    col = (i-1) % 2
    ax = axs[row, col]

    t_col = f"t{i:02d}"
    if t_col in weekly_max.columns:
        data = weekly_max[t_col].dropna()

        ax.hist(
            data,
            bins=15, alpha=0.7,
            edgecolor='black', linewidth=1.0,
            label=t_col
        )

        if not data.empty:
            max_val = data.max()
            ax.axvline(max_val, color="red", linestyle="--", linewidth=2,
                       label=f"Max = {max_val:.2f}")

        ax.set_title(f"{t_col} Weekly Max Temperature Distribution")
        ax.set_xlabel("Temperature (°C)")
        ax.set_ylabel("Frequency")
        ax.legend()

plt.suptitle("Thermistor Weekly Max Temperature Histograms: Jan2025-Sep2025", fontsize=20)
plt.tight_layout(rect=[0.05, 0, 1, 0.97])
plt.savefig(
    "/home/jovyan/data/tmpsf_dataoutputs/tmpsf_plots/thermistor_weeklymax_histograms.png",
    dpi=300, bbox_inches="tight"
)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import re

# 0) Ensure datetime index
df = df_tmpsf.copy()
df.index = pd.to_datetime(df.index, errors='coerce')

# 1) Keep only base thermistor columns like t01..t24 (exclude *_rolling etc.)
therm_cols = [c for c in df.columns if re.fullmatch(r"t\d{2}", c) and pd.api.types.is_numeric_dtype(df[c])]
df_num = df[therm_cols]

# 2) Weekly maxima (weeks start Monday)
weekly_max = df_num.resample("W-MON").max()

# 3) Plot bars: time (x) vs weekly max temp (y)
fig, axs = plt.subplots(12, 2, figsize=(20, 38), sharex=True)
locator = mdates.WeekdayLocator(byweekday=mdates.MO, interval=2)  # tick every 2 Mondays (tweak as needed)
fmt = mdates.DateFormatter('%Y-%m-%d')

for i in range(1, 25):
    row = (i-1) // 2
    col = (i-1) % 2
    ax = axs[row, col]

    t_col = f"t{i:02d}"
    if t_col in weekly_max.columns:
        s = weekly_max[t_col].dropna()
        if not s.empty:
            x = mdates.date2num(s.index)
            y = s.values

            # Bars with borders
            ax.bar(x, y, width=5, align='center', alpha=0.7,
                   edgecolor='black', linewidth=0.8, label=t_col)

            # Highlight absolute max
            imax = int(np.nanargmax(y))
            ax.scatter(x[imax], y[imax], s=60, color='red', zorder=3, label=f"Max {y[imax]:.2f}")
            ax.annotate(f"{y[imax]:.2f}", (x[imax], y[imax]),
                        xytext=(0, 6), textcoords='offset points', ha='center', fontsize=9, color='red')

        ax.set_title(f"{t_col} — Weekly Max Temps")
        if col == 0:
            ax.set_ylabel("Temperature (°C)")
        ax.xaxis.set_major_locator(locator)
        ax.xaxis.set_major_formatter(fmt)
        for label in ax.get_xticklabels():
            label.set_rotation(30)
            label.set_ha('right')
        ax.legend()

# Optional: lock y-axis for easy comparison (uncomment to use)
# for ax in axs.flat:
#     ax.set_ylim(0, 10)

plt.suptitle("Thermistor Weekly Max Temperatures Jan2025-Sep2025", fontsize=20)
plt.tight_layout(rect=[0.03, 0.02, 1, 0.97])
plt.savefig("/home/jovyan/data/tmpsf_dataoutputs/tmpsf_plots/thermistor_weeklymax_bars.png",
            dpi=300, bbox_inches="tight")
plt.show()

    